In [1]:
from vdp_python_tools.get_images import write_tile_from_api
from vdp_python_tools.authentication import login
from vdp_python_tools.tile_math import deg2num, num2deg, coord_to_pixel
from vdp_python_tools.tile_math import get_tile_boundary


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
plt.rcParams['figure.figsize'] = [10, 10]

In [3]:
token = login()

# Step 1: Inputs

There are 2 inputs:
- gdf_buildings: a GeoDataFrame containing all of the building footprints
- polygon (optional): a boundary polygon for cutting the building footprints in gdf_buildingsgdf_polygons to


In [4]:
import geopandas as gpd
import fiona
from shapely.wkt import loads as load_wkt

In [5]:
boundary = load_wkt('POLYGON ((-117.98473609781264 34.14610995587136, -117.98473609781264 34.14684027349479, -117.98473128253931 34.14693829063512, -117.98471688309304 34.147035363816805, -117.98469303814836 34.147130558172044, -117.98465997734515 34.14722295692715, -117.98461801907699 34.147311670231616, -117.98456756742495 34.14739584372781, -117.984509108266 34.147474666778955, -117.98444320459383 34.147547380275974, -117.9843704910968 34.147613283948154, -117.98429166804566 34.14767174310709, -117.98420749454947 34.14772219475914, -117.984118781245 34.1477641530273, -117.98402638248989 34.14779721383052, -117.98393118813465 34.14782105877519, -117.98383411495297 34.14783545822146, -117.98373609781264 34.147840273494786, -117.9827544093132 34.147840273494786, -117.98265639217287 34.14783545822146, -117.98255931899119 34.14782105877519, -117.98246412463595 34.14779721383052, -117.98237172588084 34.1477641530273, -117.98228301257637 34.14772219475914, -117.98219883908018 34.14767174310709, -117.98212001602904 34.147613283948154, -117.98204730253201 34.147547380275974, -117.98198139885984 34.147474666778955, -117.9819229397009 34.14739584372781, -117.98187248804885 34.147311670231616, -117.9818305297807 34.14722295692715, -117.98179746897748 34.147130558172044, -117.9817736240328 34.147035363816805, -117.98175922458653 34.14693829063512, -117.9817544093132 34.14684027349479, -117.9817544093132 34.14610995587136, -117.98175922458653 34.14601193873103, -117.9817736240328 34.145914865549344, -117.98179746897748 34.145819671194104, -117.9818305297807 34.145727272439, -117.98187248804885 34.14563855913453, -117.9819229397009 34.14555438563834, -117.98198139885984 34.145475562587194, -117.98204730253201 34.145402849090175, -117.98212001602904 34.145336945417995, -117.98219883908018 34.14527848625906, -117.98228301257637 34.14522803460701, -117.98237172588084 34.145186076338845, -117.98246412463595 34.14515301553563, -117.98255931899119 34.145129170590955, -117.98265639217287 34.14511477114469, -117.9827544093132 34.14510995587136, -117.98373609781264 34.14510995587136, -117.98383411495297 34.14511477114469, -117.98393118813465 34.145129170590955, -117.98402638248989 34.14515301553563, -117.984118781245 34.145186076338845, -117.98420749454947 34.14522803460701, -117.98429166804566 34.14527848625906, -117.9843704910968 34.145336945417995, -117.98444320459383 34.145402849090175, -117.984509108266 34.145475562587194, -117.98456756742495 34.14555438563834, -117.98461801907699 34.14563855913453, -117.98465997734515 34.145727272439, -117.98469303814836 34.145819671194104, -117.98471688309304 34.145914865549344, -117.98473128253931 34.14601193873103, -117.98473609781264 34.14610995587136))')

In [6]:
import osmnx as ox

gdf_buildings = ox.geometries_from_polygon(boundary, tags = {"building": True})
gdf_buildings = gdf_buildings.loc[gdf_buildings.within(boundary)]

In [7]:
import folium

lon, lat = boundary.centroid.coords[0]
m = folium.Map(location=[lat, lon], tiles="cartodbpositron", zoom_start=18)

folium.GeoJson(
    gdf_buildings.to_json(),
).add_to(m)
m

In [8]:
from vdp_python_tools.tile_math import tiles_in_polygon
import requests
from PIL import Image
import io
import numpy as np

def get_tile_array(x, y, z, api, layer):
    url = f"https://vdp-evaluation.herokuapp.com/{api}/{layer}/{z}/{x}/{y}"
    response = requests.get(url)
    image = Image.open(io.BytesIO(response.content))
    return np.array(image)

For each building footprint, do the following:
- for each tile that intersects it, download the tile if we haven't already
- write them down with the geospatial bounds
- read them into a sinlge raster
- crop the raster to the building
- run the QA analysis
- add a score to the BP feature

We use GeoPandas below to store the each of the points, but for more efficient results, consider using PostGIS.

In [9]:

import geopandas as gpd
import os
import shapely
from shapely.geometry import Polygon, MultiPolygon

using_postgis = False

if using_postgis:
    from sqlalchemy import create_engine
    psql_url = "postgresql://localhost:5432"
    engine = create_engine(psql_url)

In [10]:
from shapely.geometry import Point

def degrees_per_pixel(bounds, pixels_wide, pixels_high):
    width_in_deg = bounds[2] - bounds[0]
    height_in_deg = bounds[3] - bounds[1]

    deg_per_pixel_wide = width_in_deg/pixels_wide
    deg_per_pixel_high = height_in_deg/pixels_high
    return deg_per_pixel_wide, deg_per_pixel_high

def pixel_to_coord(p_x, p_y, bounds, deg_per_pixel_wide, deg_per_pixel_high):
    c_x = bounds[0] + p_x*deg_per_pixel_wide
    c_y = bounds[1] + (255-p_y)*deg_per_pixel_high
    return [ c_x, c_y]


def process_elevations(x_tile, y_tile, zoom, layer, using_postgis = False):
    dsm_array = get_tile_array(x_tile, y_tile, zoom, "GetDSMTile", layer)
    dtm_array = get_tile_array(x_tile, y_tile, zoom, "GetDTMTile", layer)
    height_array = dsm_array - dtm_array
    w, h = height_array.shape
    bounds = get_tile_boundary(x_tile, y_tile, zoom).bounds
    deg_per_pixel_wide, deg_per_pixel_high = degrees_per_pixel(bounds, w, h)
    points = []
    for x_pixel in range(w):
        for y_pixel in range(h):
            points.append([x_tile, y_tile, zoom, height_array[y_pixel, x_pixel], Point(*pixel_to_coord(x_pixel, y_pixel, bounds, deg_per_pixel_wide, deg_per_pixel_high))])


    gdf_points = gpd.GeoDataFrame(points, columns=["x_tile", "y_tile", "zoom", "height_m", "geometry"])
    gdf_points = gdf_points.set_crs(epsg=4326)
    if using_postgis:
        gdf_points.to_postgis(name=f"points_with_elevation_zoom_{zoom}", con=engine, if_exists="append")
    return gdf_points

In [11]:
if using_postgis:
    import psycopg2
    conn = psycopg2.connect()
    curs = conn.cursor()

    zoom = 16
    sql = f"create table points_with_elevation_zoom_{zoom} (id serial, name varchar, geom geometry(GEOMETRY, 4326));"
    curs.execute(sql);

In [12]:
from vdp_python_tools.tile_math import tiles_in_polygon

tiles = tiles_in_polygon(boundary, 18)

In [20]:
tiles

[(45158, 104588, 18),
 (45158, 104589, 18),
 (45159, 104588, 18),
 (45159, 104589, 18)]

In [13]:
gdfs_points = []
for x, y, zoom in tiles:
    gdfs_points.append(process_elevations(x, y, zoom, "bluesky-high", using_postgis=using_postgis))


In [14]:
import pandas as pd

gdf_points = gpd.GeoDataFrame(pd.concat(gdfs_points))

In [15]:
example_building_geometry = gdf_buildings.iloc[0].geometry

The cell below pulls out all the points within `gdf_points` that fall within the boundary of the building sampled above called `example_building_boundary`. It then plots them showing the range of elevation values across the roof of the building.

In [17]:
import branca
import os
import json

lon, lat = example_building_geometry.centroid.coords[0]

gdf_points_in_building = gdf_points.loc[gdf_points.geometry.within(example_building_geometry)]

m = folium.Map(location=[lat, lon], tiles="cartodbpositron", zoom_start=21, max_zoom=21)



geojson = json.loads(gdf_points_in_building[['height_m', 'geometry']].to_json())
colorscale = branca.colormap.linear.YlGnBu_09.scale(0, 5)

def style_function(feature):
    return {
        "fillOpacity": 0.8,
        "radius": 8,
        "weight": 0,
        "fillColor": colorscale(feature['properties']['height_m'])
    }

folium.GeoJson(
    geojson,
    marker = folium.CircleMarker(radius = 1, # Radius in metres
                                           weight = 0, #outline weight
                                           fill_color = '#000000', 
                                           fill_opacity = 1),
    style_function=style_function,
).add_to(m)

colorscale.add_to(m)
m

To keep things a bit simpler so that learning PostGIS isn't a requirement of this notebook, the function below works with a GeoPandas GeoDataFrame but is far more efficient (~100x) if you were to use a PostGIS DB to store all the point values. If you want to apply this method across large areas, it's recommended to connect to a PostGIS DB to perform the queries.

In [18]:
def calculate_max_height_in_polygon(geometry, zoom, sql_cur = None, gdf_points = None):
    if sql_cur is not None:
        sql = f"SELECT *, geometry as geom FROM points_with_elevation_zoom_{zoom} WHERE ST_within(geometry, ST_GeomFromText('{geometry.wkt}', 4326))"
        curs.execute(sql)
        return max([r[3] for r in curs])
    elif gdfs_points is not None:
        gdf_points_in_building = gdf_points.loc[gdf_points.geometry.within(geometry)]
        return gdf_points_in_building.height_m.max()
    raise ValueError("Must supply at sql cursor object or GeoDataFrame with elevation points")
    
gdf_buildings['height_m'] = gdf_buildings.geometry.apply(lambda geometry: calculate_max_height_in_polygon(geometry, 19, gdf_points=gdf_points))

In [19]:

lon, lat = boundary.centroid.coords[0]

m = folium.Map(location=[lat, lon], tiles="cartodbpositron", zoom_start=21, max_zoom=21)

geojson = json.loads(gdf_buildings[['height_m', 'geometry']].to_json())
colorscale = branca.colormap.linear.YlGnBu_09.scale(0, 5)

def style_function(feature):
    return {
        "fillOpacity": 0.8,
        "radius": 8,
        "weight": 0,
        "fillColor": colorscale(feature['properties']['height_m'] if feature['properties']['height_m'] else 0)
    }

folium.GeoJson(
    geojson,
    style_function=style_function,
).add_to(m)

colorscale.add_to(m)
m